In [2]:
# generate an nlp model to cluster 21 different appartments based on descriptions and slightly fluctauting prices

# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from unidecode import unidecode
from num2words import num2words

In [3]:
# import data
jan_apr_desc = pd.read_csv('dados_afonso/room_say_report/reservas_por_data_criada/jan_apr.csv')
may_june_desc = pd.read_csv('dados_afonso/room_say_report/reservas_por_data_criada/may_june.csv')
jul_aug_desc = pd.read_csv('dados_afonso/room_say_report/reservas_por_data_criada/jul_aug.csv')

data_desc = pd.concat([jan_apr_desc, may_june_desc, jul_aug_desc])

jan_mar_pri = pd.read_csv('dados_afonso/reservation_summary/reservas_por_data_criada/jan_mar.csv')
apr_jun_pri = pd.read_csv('dados_afonso/reservation_summary/reservas_por_data_criada/apr_jun.csv')
jul_set_pri = pd.read_csv('dados_afonso/reservation_summary/reservas_por_data_criada/jul_set.csv')
aug_pri = pd.read_csv('dados_afonso/reservation_summary/reservas_por_data_criada/aug.csv')

data_pri = pd.concat([jan_mar_pri, apr_jun_pri, jul_set_pri, aug_pri])


In [4]:
data = pd.merge(data_desc, data_pri, on='Reservation ID', how='left')

data = data[data['Channel_x'] == 'Booking.com']

In [5]:
data_cols = data[['Reservation ID', 'Room', 'Total Amount']]

In [6]:
data_cols.rename(columns={'Reservation ID' : 'id', 'Room' : 'desc', 'Total Amount': 'price'}, inplace=True)

/tmp/ipykernel_3203/2440387659.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cols.rename(columns={'Reservation ID' : 'id', 'Room' : 'desc', 'Total Amount': 'price'}, inplace=True)


In [7]:
data_cols.price = data_cols.price.str[:-5]

/tmp/ipykernel_3203/1562624752.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cols.price = data_cols.price.str[:-5]


In [8]:
data_cols.price = data_cols.price.astype('float')

/tmp/ipykernel_3203/1542369163.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cols.price = data_cols.price.astype('float')


In [9]:
# use nlp to cluster appartments based on descriptions

# clean data but leave numbers
data_cols = data_cols.drop_duplicates(subset='id', keep=False)
data = data.drop_duplicates(subset='Reservation ID', keep=False)

def clean_text(text):
    text = unidecode(text)
    text = text.lower()
    return text

data_cols['desc'] = data_cols['desc'].apply(clean_text)

# select number before "quarto" and make it a column
def add_n_rooms(text):
    n_rooms = re.findall(r'(\d+)\s+quarto', text)
    if len(n_rooms) > 0:
        return int(n_rooms[0])
    else:
        return 1
    
data_cols['n_rooms'] = data_cols['desc'].apply(add_n_rooms)

# add a columns for road
def add_road(text):
    rua = re.findall(r'rua\s+(\w+)', text)
    if len(rua) > 0:
        return rua[0]
    else:
        return 'no_road'
    
data_cols['road'] = data_cols['desc'].apply(add_road)

# remove stuff that I've noticed is not relevant
def remove_manually(text):
    text = re.sub(r'\d{1,2}%', '', text)
    text = text.replace('nao reembolsavel', '')
    text = text.replace('()', '')
    text = text.replace('-', '')
    text = text.replace('gener', '')
    text = text.replace('oferta', '')
    text = text.replace('apartamento', '')
    text = text.replace('com', '')
    text = text.replace('high season', '')
    text = text.replace('night', '')
    text = text.replace('reserva antecipada', '')
    text = re.sub(r'\d{1,2} adultos', '', text)
    text = re.sub(r'\d{1,2} de \w{3} de', '', text)
    text = re.sub(r'\w{3} \d{1,2}', '', text)
    text = re.sub(r'\d', '', text)
    return text


data_cols['desc'] = data_cols['desc'].apply(remove_manually)


# lemmatize data
lemmatizer = WordNetLemmatizer()
def lemmatize_text(text):
    text = [lemmatizer.lemmatize(word) for word in text.split()]
    return " ".join(text)

data_cols['desc'] = data_cols['desc'].apply(lemmatize_text)

# stem data
stemmer = PorterStemmer()
def stem_text(text):
    text = [stemmer.stem(word) for word in text.split()]
    return " ".join(text)

data_cols['desc'] = data_cols['desc'].apply(stem_text)

# remove stopwords
stop_words = set(stopwords.words('portuguese'))

def remove_stopwords(text):
    text = [word for word in text.split() if word not in stop_words]
    return " ".join(text)

data_cols['desc'] = data_cols['desc'].apply(remove_stopwords)

# vectorize data
vectorizer = TfidfVectorizer()

# one hot encode road
road_ohe = pd.get_dummies(data_cols['road'])

# define X, which contains a vector for each description and a price
vectors = vectorizer.fit_transform(data_cols['desc'])
vectors_df = pd.DataFrame(vectors.toarray())
price = data_cols['price']
n_rooms = data_cols['n_rooms']
road_ohe = pd.get_dummies(data_cols['road'])
X = np.column_stack((vectors.toarray(), n_rooms, road_ohe))


# cluster data
kmeans_room = KMeans(n_clusters=21, random_state=0).fit(X)
kmeans_type = KMeans(n_clusters=14, random_state=0).fit(X)

# add cluster column to data
data_cols['cluster_room'] = kmeans_room.labels_
data_cols['cluster_type'] = kmeans_type.labels_

# add cluster column to data
data['cluster_room'] = kmeans_room.labels_
data['cluster_type'] = kmeans_type.labels_


In [10]:
data_cols.head(10)


,id,desc,price,n_rooms,road,cluster_room,cluster_type
0,2468606756,piso terreo inacio,82.08,1,no_road,9,9
1,3404160971,"quarto () rua remedios, inacio",123.12,1,dos,14,13
2,3545920362,"quarto rua santo estevao, s",379.62,1,de,2,2
3,3807966157,piso terreo s,179.82,1,no_road,9,9
4,2769935846,"quarto () rua remedios, n.o s",390.94,2,dos,6,0
5,3554122759,quarto al s,253.08,1,no_road,12,8
6,3554197542,"quarto () rua remedios, n.o limitedtim deal",145.08,2,dos,6,0
7,2719193349,"quarto () rua remedios, n.o limitedtim deal",199.02,2,dos,6,0
8,2841079705,"quarto rua remedios, limitedtim deal",564.20,1,dos,14,13
9,3086509714,"superior quarto rua remedios, geniusgeor",180.88,1,dos,0,13


In [11]:
data_cols.id.nunique()

1388

In [12]:
data_read = data[['Room', 'cluster_room', 'cluster_type']]
sample = data_read.sample(1)

print(f"{sample['Room']} is room {sample['cluster_room']} which is in cluster {sample['cluster_type']}")


117    Apartamento com 1 Quarto - General - High Seas...
Name: Room, dtype: object is room 117    12
Name: cluster_room, dtype: int32 which is in cluster 117    8
Name: cluster_type, dtype: int32


In [13]:
data_cols.groupby('cluster_room').cluster_type.nunique()

cluster_room
0     2
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
12    1
13    1
14    1
15    2
16    1
17    1
18    1
19    2
20    4
Name: cluster_type, dtype: int64

In [22]:
data_cols[data_cols.cluster_room == 15][['desc', 'n_rooms']]

,desc,n_rooms
90,"quarto () rua remedios, n.o unit state po",2
219,"quarto () rua remedios, n.o al unit state po",2
283,"quarto () rua remedios, n.o al unit state po",2
447,"quarto () rua remedios, al unit state po",1
577,"quarto () rua remedios, n.o al unit state po",2
803,"quarto () rua remedios, al unit state po",1
880,"quarto () rua remedios, n.o al unit state po",2
896,"superior quarto rua remedios, al unit state po",1
928,"superior quarto rua remedios, al unit state po",1
933,"superior quarto rua remedios, al unit state po",1


In [25]:
data_cols.groupby('cluster_room').n_rooms.nunique()

cluster_room
0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
12    1
13    1
14    1
15    2
16    1
17    1
18    1
19    1
20    1
Name: n_rooms, dtype: int64

In [158]:
texto

'Apartamento com 2 Quartos e Vista Rio - Escadinhas de Santo Estê - General - Canada POS'

In [30]:
data.Room[data.cluster_room == 15].values

array(['Apartamento com 2 Quartos (4 Adultos) - Rua dos Remédios, n.º 30 - United States POS',
       'Apartamento com 2 Quartos (4 Adultos) - Rua dos Remédios, n.º 30 - General - United States POS',
       'Apartamento com 2 Quartos (4 Adultos) - Rua dos Remédios, n.º 30 - General - United States POS',
       'Apartamento com 1 Quarto (2 Adultos) - Rua dos Remédios, 30 - General - United States POS',
       'Apartamento com 2 Quartos (4 Adultos) - Rua dos Remédios, n.º 30 - General - United States POS',
       'Apartamento com 1 Quarto (2 Adultos) - Rua dos Remédios, 30 - General - United States POS',
       'Apartamento com 2 Quartos (4 Adultos) - Rua dos Remédios, n.º 30 - General - United States POS',
       'Apartamento Superior com 1 Quarto - Rua dos Remédios, 3 - General - United States POS',
       'Apartamento Superior com 1 Quarto - Rua dos Remédios, 3 - General - United States POS',
       'Apartamento Superior com 1 Quarto - Rua dos Remédios, 3 - General - United States POS